In [1]:
import os
import tensorflow as tf
# Tải các mô hình nén từ tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import tensorflow_hub as hub
import tensorflow as tf

import PIL.Image
import time
import functools

In [18]:
class neuran_style_tranfer:
  def __init__ (self,content_path,style_path):
    self.content_image = self.load_img(content_path)
    self.style_image = self.load_img(style_path)

    content_layers = ['block5_conv2'] 

    style_layers = ['block1_conv1',
                    'block2_conv1',
                    'block3_conv1', 
                    'block4_conv1', 
                    'block5_conv1']

    self.num_content_layers = len(content_layers)
    self.num_style_layers = len(style_layers)

    self.style_extractor = self.vgg_layers(style_layers)
    self.style_outputs = self.style_extractor(self.style_image*255)   

    self.extractor = self.StyleContentModel(style_layers, content_layers)

    self.results = self.extractor(tf.constant(self.content_image)) 

    self.style_targets = self.extractor(self.style_image)['style'] 
    self.content_targets = self.extractor(self.content_image)['content']

    self.image = tf.Variable(self.content_image)

    self.opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    self.style_weight=1e-2
    self.content_weight=1e4  
    self.StyleContentModel = self.StyleContentModel()
  def tensor_to_image(self,tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
      assert tensor.shape[0] == 1
      tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

  def load_img(self, path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    self.img = img[tf.newaxis, :]
    return self.img

  def vgg_layers(self, layer_names):
    """ Creates a VGG model that returns a list of intermediate output values."""
    # Load our model. Load pretrained VGG, trained on ImageNet data
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    
    outputs = [vgg.get_layer(name).output for name in layer_names]

    self.model = tf.keras.Model([vgg.input], outputs)
    return self.model

  def gram_matrix(sefl,input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

  def style_content_loss(self,outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-self.style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= self.style_weight / self.num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-self.content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= self.content_weight / self.num_content_layers
    self.loss = style_loss + content_loss
    return self.loss

  class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
      self.vgg = neuran_style_tranfer.vgg_layers(layer_names= (style_layers + content_layers))
      self.style_layers = style_layers
      self.content_layers = content_layers
      self.num_style_layers = len(style_layers)
      self.vgg.trainable = False

    def call(self, inputs):
      "Expects float input in [0,1]"
      inputs = inputs*255.0
      preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
      outputs = self.vgg(preprocessed_input)
      style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                        outputs[self.num_style_layers:])

      style_outputs = [self.gram_matrix(style_output)
                      for style_output in style_outputs]

      self.content_dict = {content_name: value
                      for content_name, value
                      in zip(self.content_layers, content_outputs)}

      self.style_dict = {style_name: value
                    for style_name, value
                    in zip(self.style_layers, style_outputs)}

      return {'content': self.content_dict, 'style': self.style_dict}

  def clip_0_1(self,image):
    return tf.clip_by_value(self.image, clip_value_min=0.0, clip_value_max=1.0)

  @tf.function()
  def train_step(self,image):
    # tính đạo hàm
    with tf.GradientTape() as tape:
      # Tìm giá trị dự đoán và tính giá trị tổn thất
      outputs = self.extractor(image)
      loss = self.style_content_loss(outputs)
  #Tính toán độ dốc
    grad = tape.gradient(loss, image) 
    #Yêu cầu trình tối ưu hóa áp dụng các chuyển màu đã xử lý.
    self.opt.apply_gradients([(grad, image)])
    # grad: độ dốc; image: biến
    # Trả lại:Một Operation áp dụng các gradient được chỉ định. Các lần lặp sẽ được tự động tăng thêm 1.
    self.image.assign(self.clip_0_1(image)) # cập nhật

  def sanpham(self):
    epochs = 10
    steps_per_epoch = 100

    for n in range(epochs):
      for m in range(steps_per_epoch):
        self.train_step(self.image)
      display.clear_output(wait=True)
      display.display(self.tensor_to_image(self.image))

  def return_img(self):
    return self.tensor_to_image(self.image)

In [19]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [20]:
content_path = tf.keras.utils.get_file('d3305f1a76111419459a883a11202fac.jpg','https://i.ibb.co/dJ6gYSv/d3305f1a76111419459a883a11202fac.jpg')
style_path = tf.keras.utils.get_file('test.jpg','https://i.ibb.co/PmdRzKZ/test.jpg')

In [21]:
a = neuran_style_tranfer(content_path,style_path)

TypeError: ignored